# 循环神经网络语言模型
## 自然语言编码
神经网络无法直接处理汉字，需要将汉字编号。下面这段代码就是利用Python的字典，对一句话中的每个字进行编号。


In [1]:
def encode_sentence(s,chars):
    sid=[0]
    for c in s:
        if not c in chars:
            chars[c]=len(chars)
        sid.append(chars[c])
    sid.append(1)
    return sid

In [2]:
chars={'<BOS>':0,'<EOS>':1,'<UNK>':2}
sen="巴黎是法国的首都及最大都市，同时是法兰西岛大区首府，为法国的政治与文化中心，隶属法兰西岛大区之下的巴黎省"
encode_sentence(sen,chars)


[0,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 9,
 12,
 13,
 3,
 14,
 15,
 16,
 17,
 18,
 6,
 19,
 20,
 6,
 21,
 15,
 3,
 22,
 23,
 9,
 24,
 25,
 3,
 26,
 27,
 6,
 21,
 15,
 3,
 28,
 29,
 30,
 31,
 32,
 3,
 33,
 32,
 9,
 34,
 35,
 9,
 10,
 11,
 9,
 12,
 13,
 3,
 36,
 37,
 38,
 39,
 40,
 3,
 41,
 14,
 3,
 26,
 27,
 3,
 32,
 42,
 6,
 19,
 20,
 3,
 42,
 24,
 30,
 31,
 32,
 43,
 28,
 42,
 9,
 12,
 13,
 3,
 14,
 15,
 16,
 17,
 18,
 9,
 44,
 40,
 9,
 41,
 7,
 43,
 28,
 8,
 9,
 20,
 45,
 3,
 32,
 20,
 43,
 28,
 46,
 3,
 40,
 21,
 30,
 31,
 32,
 6,
 17,
 35,
 3,
 47,
 48,
 9,
 12,
 13,
 3,
 36,
 37,
 38,
 39,
 40,
 3,
 41,
 14,
 3,
 26,
 27,
 3,
 32,
 42,
 43,
 49,
 50,
 43,
 28,
 50,
 16,
 17,
 18,
 3,
 4,
 5,
 6,
 7,
 8,
 16,
 24,
 51,
 1]

## 读取数据
读取的同时将汉字处理成上述的编号，同时要记录汉字和编号的对应表

In [3]:
import os
import json
import pickle

def prepare_data(dir_):
    chars={'<BOS>':0,'<EOS>':1,'<UNK>':2}
    sentences=[]
    sids=[]
    files=os.listdir(dir_)
    for file_ in files:
        al=os.path.join(dir_,file_)
        print al
        with open(al,'r') as f:
            lines=f.readlines()
            for line in lines:
                data=json.loads(line)
                text=data['text']
                sen=text.split('\n')
                for s in sen:
                    if len(s.strip())>0:
                        sentences.append(s)
                        sid=encode_sentence(s,chars)
                        sids.append(sid)
    n_char=len(chars)
    print 'vocabulary_size=%d'%n_char
    pickle.dump(chars,open('chars.pkl','wb'))
    
    return sentences,sids,chars

In [4]:
sentences,sids,chars=prepare_data("corpus")

corpus/wiki_81
corpus/wiki_65
corpus/wiki_64
corpus/wiki_22
corpus/wiki_92
corpus/wiki_48
corpus/wiki_37
corpus/wiki_86
corpus/wiki_85
corpus/wiki_14
corpus/wiki_70
corpus/wiki_55
corpus/wiki_99
corpus/wiki_09
corpus/wiki_11
corpus/wiki_84
corpus/wiki_53
corpus/wiki_44
corpus/wiki_17
corpus/wiki_60
corpus/wiki_74
corpus/wiki_25
corpus/wiki_80
corpus/wiki_50
corpus/wiki_30
corpus/wiki_68
corpus/wiki_73
corpus/wiki_67
corpus/wiki_26
corpus/wiki_10
corpus/wiki_82
corpus/wiki_38
corpus/wiki_20
corpus/wiki_12
corpus/wiki_47
corpus/wiki_96
corpus/wiki_94
corpus/wiki_43
corpus/wiki_45
corpus/wiki_66
corpus/wiki_04
corpus/wiki_08
corpus/wiki_83
corpus/wiki_77
corpus/wiki_35
corpus/wiki_27
corpus/wiki_87
corpus/wiki_13
corpus/wiki_07
corpus/wiki_46
corpus/wiki_29
corpus/wiki_71
corpus/wiki_16
corpus/wiki_54
corpus/wiki_62
corpus/wiki_63
corpus/wiki_69
corpus/wiki_90
corpus/wiki_24
corpus/wiki_05
corpus/wiki_34
corpus/wiki_72
corpus/wiki_00
corpus/wiki_23
corpus/wiki_98
corpus/wiki_91
corpus/wik

## 训练神经网络
首先设置一些超参数

In [5]:
class Args(object):
    max_length=64
    n_emb=128
    vocab_size=12000
    n_hidden=256
    batch_size=128

## 开始训练流程

In [8]:
from lstm import LSTMLM
import numpy as np
import copy

def train(sids):
    args=Args()
    lstm=LSTMLM(args)
    lstm.build_model()
    
    for i in range(40000):
        batch_sen=np.random.choice(sids,size=args.batch_size)
        batch_sen=[copy.copy(s) for s in batch_sen]
        loss=lstm.train(batch_sen)
        if i%10==0:
            print 'step=%d, loss=%.3f'%(i,loss)
        if i%1000==0 and i!=0:
            lstm.save_model('model')

train(sids)

step=0, loss=9.393
step=10, loss=7.048
step=20, loss=6.860
step=30, loss=6.818
step=40, loss=6.783
step=50, loss=6.719
step=60, loss=6.580
step=70, loss=6.470
step=80, loss=6.415
step=90, loss=6.422
step=100, loss=6.247
step=110, loss=6.280
step=120, loss=6.177
step=130, loss=6.166
step=140, loss=6.063
step=150, loss=6.022
step=160, loss=5.977
step=170, loss=5.795
step=180, loss=5.753
step=190, loss=5.676
step=200, loss=5.671
step=210, loss=5.646
step=220, loss=5.481
step=230, loss=5.556
step=240, loss=5.490
step=250, loss=5.213
step=260, loss=5.177
step=270, loss=5.127
step=280, loss=5.059
step=290, loss=5.099
step=300, loss=5.112
step=310, loss=5.171
step=320, loss=5.010
step=330, loss=5.043
step=340, loss=5.038
step=350, loss=4.899
step=360, loss=4.961
step=370, loss=4.906
step=380, loss=4.886
step=390, loss=4.858
step=400, loss=4.875
step=410, loss=4.848
step=420, loss=4.810
step=430, loss=4.884
step=440, loss=4.864
step=450, loss=4.956
step=460, loss=4.869
step=470, loss=4.746
ste

KeyboardInterrupt: 

## 测试和使用
语言模型可以判断任意字符串是自然语言的概率，有非常多的用处。
### 判断几句话中哪句更通顺

In [70]:
from lstm import LSTMLM
import numpy as np

def get_prob(sen):
    sen=sen.decode('utf-8')
    args=Args()
    lstm=LSTMLM(args)
    lstm.build_model()
    lstm.load_model('model')
    chars=pickle.load(open('chars.pkl','rb'))
    
    prob=0.
    segments=[c for c in sen]
    segments.insert(0,'<BOS>')
    segments.append('<EOS>')
    sid=[(chars[c] if c in chars else 2)for c in segments]
    eprob=0
    for i in range(1,len(sid)):
        dist=lstm.next_char([sid[:i]])[0]
        eprob+=np.log(sid[i])
        epp=-eprob/i
        print sen[:i].encode('utf-8'),epp,eprob
    return epp


In [72]:
sen1="分哈啊词腌可"
print get_prob(sen1)
sen2="数学是一门历史悠久的学科。"
print get_prob(sen2)

Load model model/model.ckpt-5000
INFO:tensorflow:Restoring parameters from model/model.ckpt-5000
分 -5.7745515455444085 5.7745515455444085
分哈 -6.1164449144446 12.2328898288892
分哈啊 -6.763105532137395 20.289316596412185
分哈啊词 -6.91107962937936 27.64431851751744
分哈啊词腌 -7.194312224995047 35.97156112497523
分哈啊词腌可 -7.021776085577861 42.130656513467166
分哈啊词腌可 -6.0186652162095955 42.130656513467166
-6.0186652162095955
Load model model/model.ckpt-5000
INFO:tensorflow:Restoring parameters from model/model.ckpt-5000
数 -4.51085950651685 4.51085950651685
数学 -5.001960474928699 10.003920949857399
数学是 -4.819422748703635 14.458268246110906
数学是一 -4.680237030788056 18.720948123152223
数学是一门 -5.2011538071441645 26.005769035720824
数学是一门历 -5.3622142544348605 32.17328552660916
数学是一门历史 -5.477556608442946 38.34289625910062
数学是一门历史悠 -5.766905647901707 46.135245183213655
数学是一门历史悠久 -5.924177477233213 53.31759729509892
数学是一门历史悠久的 -5.732493048033139 57.32493048033139
数学是一门历史悠久的学 -5.7107265385156305 62.81799192367194
数

### 将句子补齐

In [110]:
from lstm import LSTMLM
import numpy as np
import copy

def maximum_generate(prefix):
    prefix=prefix.decode('utf-8')
    args=Args()
    lstm=LSTMLM(args)
    lstm.build_model()
    lstm.load_model('model')
    chars=pickle.load(open('chars.pkl','rb'))
    rchars={chars[c]:c for c in chars}
    
    segments=[c for c in prefix]
    segments.insert(0,'<BOS>')
    sid=[(chars[c] if c in chars else 2)for c in segments]
    str_=prefix
    while sid[-1]!=1 and len(sid)<64:
        dist=lstm.next_char([copy.copy(sid)])[0]
        nxt=np.random.choice(range(args.vocab_size),p=dist)
        sid.append(nxt)
        c2=rchars[nxt]
        str_+=c2
        print str_.encode('utf-8')
    return str_

In [111]:
print maximum_generate("数学").encode('utf-8')

Load model model/model.ckpt-5000
INFO:tensorflow:Restoring parameters from model/model.ckpt-5000
数学上
数学上对
数学上对代
数学上对代表
数学上对代表f
数学上对代表fo
数学上对代表for
数学上对代表fort
数学上对代表forte
数学上对代表fortec
数学上对代表fortec导
数学上对代表fortec导有
数学上对代表fortec导有，
数学上对代表fortec导有，化
数学上对代表fortec导有，化合
数学上对代表fortec导有，化合物
数学上对代表fortec导有，化合物虚
数学上对代表fortec导有，化合物虚拟
数学上对代表fortec导有，化合物虚拟（
数学上对代表fortec导有，化合物虚拟（明
数学上对代表fortec导有，化合物虚拟（明永
数学上对代表fortec导有，化合物虚拟（明永元
数学上对代表fortec导有，化合物虚拟（明永元接
数学上对代表fortec导有，化合物虚拟（明永元接管
数学上对代表fortec导有，化合物虚拟（明永元接管)
数学上对代表fortec导有，化合物虚拟（明永元接管)，
数学上对代表fortec导有，化合物虚拟（明永元接管)，为
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了C
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了CD
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了CD的
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了CD的结
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了CD的结构
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了CD的结构。
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了CD的结构。另
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了CD的结构。另外
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了CD的结构。另外隔
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了CD的结构。另外隔离
数学上对代表fortec导有，化合物虚拟（明永元接管)，为了CD的结构。另外隔离两
数学上对代表